In [10]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
import ast
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf

import keras
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from keras.optimizers import Adam
from keras.applications import MobileNet
from keras.applications import ResNet50
from keras.applications import InceptionResNetV2
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input
# from keras.applications.inception_resnet_v2 import preprocess_input
# from keras.applications.resnet50 import preprocess_input
# from keras.applications.mobilenet import preprocess_input
from keras.utils.training_utils import multi_gpu_model
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.preprocessing.image import load_img
from keras import Model
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.merge import concatenate
from keras.layers import GlobalAveragePooling2D, Reshape, Dense, multiply, Permute, Input
from keras.layers import UpSampling2D, Conv2D, Conv2DTranspose, Concatenate, MaxPooling2D
from keras.layers import concatenate, Dropout, BatchNormalization, Lambda, Flatten, Activation
from keras.losses import binary_crossentropy
from keras import backend as K

def step_decay_exp(epoch):
    initial_lrate = 0.01
    drop = 0.5 # drop by how much? 0.5 = half each time
    epochs_drop = 10.0 # drop how many x epochs
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

def cse_block(input, prefix, ratio=16):
    ''' Create a squeeze-excite block
    Args:
        input: input tensor
        filters: number of output filters
        k: width factor
    Returns: a keras tensor
    '''
    init = input
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = init._keras_shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D(name = prefix + "_cse_gap1")(init)
    se = Reshape(se_shape, name = prefix + "_cse_reshape1")(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False,
              name = prefix + "_cse_fc1")(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False,
              name = prefix + "_cse_fc2")(se)

    if K.image_data_format() == 'channels_first':
        se = Permute((3, 1, 2))(se)

    x = multiply([init, se], name = prefix + "_cse_mul1")
    return x

def sse_block(input, prefix):
    init = input
    conv = Conv2D(1, (1,1), padding = 'same', kernel_initializer = 'he_normal',
                  activation = 'sigmoid', strides = (1,1), name = prefix + "_sse_conv")(input)
    conv = Multiply(name = prefix + "_sse_mul1")([input, conv])
    return conv

def csse_block(x, prefix):
    cse = cse_block(x, prefix)
    sse = sse_block(x, prefix)
    out = Add(name = prefix + "_csse_mul")([cse, sse])
    return out

### Kaggle Code
def apk(actual, predicted, k=3):
    """
    Source: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
    """
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=3):
    """
    Source: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
    """
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

def preds2catids(predictions, k=3):
    return pd.DataFrame(np.argsort(-predictions, axis=1)[:, :k], columns=[chr(ord('a')+x) for x in range(k)])

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def f2cat(filename: str) -> str:
    return filename.split('.')[0]

def list_all_categories():
    files = os.listdir(os.path.join(INPUT_DIR, 'train_simplified'))
    return sorted([f2cat(f) for f in files], key=str.lower)

def multi_gpu_model(model, gpus):
    if gpus == 1:
        return model
    if isinstance(gpus, (list, tuple)):
        num_gpus = len(gpus)
        target_gpu_ids = gpus
    else:
        num_gpus = gpus
        target_gpu_ids = range(num_gpus)

    def get_slice(data, i, parts):
        shape = tf.shape(data)
        batch_size = shape[:1]
        input_shape = shape[1:]
        step = batch_size // parts
        if i == num_gpus - 1:
            size = batch_size - step * i
        else:
            size = step
        size = tf.concat([size, input_shape], axis=0)
        stride = tf.concat([step, input_shape * 0], axis=0)
        start = stride * i
        return tf.slice(data, start, size)

    all_outputs = []
    for i in range(len(model.outputs)):
        all_outputs.append([])

    # Place a copy of the model on each GPU,
    # each getting a slice of the inputs.
    for i, gpu_id in enumerate(target_gpu_ids):
        with tf.device('/gpu:%d' % gpu_id):
            with tf.name_scope('replica_%d' % gpu_id):
                inputs = []
                # Retrieve a slice of the input.
                for x in model.inputs:
                    input_shape = tuple(x.get_shape().as_list())[1:]
                    slice_i = Lambda(get_slice,output_shape=input_shape,arguments={'i': i, 'parts': num_gpus})(x)
                    inputs.append(slice_i)

                # Apply model on slice
                # (creating a model replica on the target device).
                outputs = model(inputs)
                if not isinstance(outputs, list):
                    outputs = [outputs]

                # Save the outputs for merging back together later.
                for o in range(len(outputs)):
                    all_outputs[o].append(outputs[o])

    # Merge outputs on CPU.
    with tf.device('/cpu:0'):
        merged = []
        for name, outputs in zip(model.output_names, all_outputs):
            merged.append(concatenate(outputs, axis=0, name=name))
        return Model(model.inputs, merged)

### Image Generator
def draw_cv2(raw_strokes, size=256, lw=6, time_color=True):
    img = np.zeros((BASE_SIZE, BASE_SIZE), np.uint8)
    for t, stroke in enumerate(raw_strokes):
        for i in range(len(stroke[0]) - 1):
            # can try to do exponential smoothing / decay
            # how about embedding different elements/colours to different channels
            # one channel has raw 0/255
            # one channel has decay 0->255
            # one channel has decay 0->255 but with different line width
            color = 255 - min(t, 15) * 13 if time_color else 255
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]),
                         (stroke[0][i + 1], stroke[1][i + 1]), color, lw)
    if size != BASE_SIZE:
        return cv2.resize(img, (size, size))
    else:
        return img

def image_generator_xd(size, batchsize, ks, lw=6, time_color=True):
    while True:
        for k in np.random.permutation(ks):
#             print('Chosen {}'.format(k))
            filename = os.path.join(DP_DIR, 'train_k{}.csv.gz'.format(k))
            for df in pd.read_csv(filename, chunksize=batchsize):
                df['drawing'] = df['drawing'].apply(ast.literal_eval)
                x = np.zeros((len(df), size, size, 3))
                for i, raw_strokes in enumerate(df.drawing.values):
                    x[i, :, :, 0] = draw_cv2(raw_strokes, size=size, lw=lw,
                                             time_color=time_color)
                for i, raw_strokes in enumerate(df.drawing.values):
                    x[i, :, :, 1] = draw_cv2(raw_strokes, size=size, lw=lw,
                                             time_color=False)
                for i, raw_strokes in enumerate(df.drawing.values):
                    x[i, :, :, 2] = draw_cv2(raw_strokes, size=size, lw=4,
                                             time_color=True)
#                 x = np.repeat(x, 3, axis = 3)
                x = preprocess_input(x).astype(np.float32)
                y = keras.utils.to_categorical(df.y, num_classes=NCATS)
                yield x, y

def df_to_image_array_xd(df, size, lw=6, time_color=True):
    df['drawing'] = df['drawing'].apply(ast.literal_eval)
    x = np.zeros((len(df), size, size, 3))
    for i, raw_strokes in enumerate(df.drawing.values):
        x[i, :, :, 0] = draw_cv2(raw_strokes, size=size, lw=lw, time_color=time_color)
    for i, raw_strokes in enumerate(df.drawing.values):
        x[i, :, :, 1] = draw_cv2(raw_strokes, size=size, lw=lw, time_color=False)
    for i, raw_strokes in enumerate(df.drawing.values):
        x[i, :, :, 2] = draw_cv2(raw_strokes, size=size, lw=4, time_color=True)
    x = preprocess_input(x).astype(np.float32)
    return x

In [2]:
DP_DIR = '/home/wilan/Kaggle/shuffled_train_20/'
INPUT_DIR = '/home/wilan/Kaggle/'

BASE_SIZE = 256
NCSVS = 20
NCATS = 340
np.random.seed(seed=123)
tf.set_random_seed(seed=123)
size = 128

In [21]:
Xception(input_shape = (128, 128, 3), include_top = False, weights = None).summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 63, 63, 32)   864         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 63, 63, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 63, 63, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [4]:
base_model = Xception(input_shape = (128, 128, 3), include_top = False, weights = 'imagenet')
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(340, activation = 'softmax')(x)
model = Model(inputs = base_model.input, outputs = x)
gpu_model = multi_gpu_model(model, 2)
print(gpu_model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 128, 128, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 128, 128, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
model_3 (Model)                 (None, 340)          21558140    lambda_3[0][0]                   
                                                                 lambda_4[0][0]                   
__________

In [22]:
### Xception
from keras.callbacks import CSVLogger

EPOCHS = 100
TOTAL_TRAIN = 2250000
BATCH_SIZE = 128 * 2
STEPS = TOTAL_TRAIN // BATCH_SIZE

train_datagen = image_generator_xd(size=size, batchsize=BATCH_SIZE, ks=range(NCSVS - 1))
valid_datagen = image_generator_xd(size=size, batchsize=BATCH_SIZE, ks=NCSVS - 1)

def step_decay_exp(epoch):
    initial_lrate = 0.00075
    drop = 0.5 # drop by how much? 0.5 = half each time
    epochs_drop = 7.0 # drop how many x epochs
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return max(1e-6, lrate)

callbacks = [
    ModelCheckpoint('xception_model.h5', monitor = 'val_top_3_accuracy', mode = 'max', save_best_only = True, verbose = 1),
    ModelCheckpoint('xception_model_last.h5', monitor = 'val_top_3_accuracy', mode = 'max', save_best_only = False, verbose = 0),
    ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.66, patience=3,
                      min_lr = 0.000001, 
                      mode='max', 
                      cooldown=3, verbose=1),
#     LearningRateScheduler(step_decay_exp),
    CSVLogger('xception_log.csv', append=True, separator=';')
]

# sgdOptimiser = SGD(lr = 0.001, momentum=0.9, decay = 0.0001,  nesterov=True)

gpu_model.compile(optimizer=Adam(lr=0.00075), loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])

gpu_model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data = valid_datagen, validation_steps = STEPS // 13, # can try STEPS // 6 or 7 for more consistent validation
    callbacks = callbacks
)

Epoch 1/100
8789/8789 [==============================] - 4697s 534ms/step - loss: 1.0701 - categorical_crossentropy: 1.0701 - categorical_accuracy: 0.7263 - top_3_accuracy: 0.8838 - val_loss: 0.9115 - val_categorical_crossentropy: 0.9115 - val_categorical_accuracy: 0.7637 - val_top_3_accuracy: 0.9081

Epoch 00001: val_top_3_accuracy improved from -inf to 0.90813, saving model to xception_model.h5
Epoch 2/100
8789/8789 [==============================] - 4673s 532ms/step - loss: 0.8684 - categorical_crossentropy: 0.8684 - categorical_accuracy: 0.7735 - top_3_accuracy: 0.9130 - val_loss: 0.8406 - val_categorical_crossentropy: 0.8406 - val_categorical_accuracy: 0.7815 - val_top_3_accuracy: 0.9170

Epoch 00002: val_top_3_accuracy improved from 0.90813 to 0.91703, saving model to xception_model.h5
Epoch 3/100
8789/8789 [==============================] - 4690s 534ms/step - loss: 0.8126 - categorical_crossentropy: 0.8126 - categorical_accuracy: 0.7877 - top_3_accuracy: 0.9202 - val_loss: 0.795

8789/8789 [==============================] - 4697s 534ms/step - loss: 0.5938 - categorical_crossentropy: 0.5938 - categorical_accuracy: 0.8414 - top_3_accuracy: 0.9469 - val_loss: 0.5676 - val_categorical_crossentropy: 0.5676 - val_categorical_accuracy: 0.8471 - val_top_3_accuracy: 0.9497

Epoch 00022: val_top_3_accuracy did not improve from 0.95508
Epoch 23/100
8789/8789 [==============================] - 4703s 535ms/step - loss: 0.6051 - categorical_crossentropy: 0.6051 - categorical_accuracy: 0.8390 - top_3_accuracy: 0.9455 - val_loss: 0.5634 - val_categorical_crossentropy: 0.5634 - val_categorical_accuracy: 0.8473 - val_top_3_accuracy: 0.9502

Epoch 00023: val_top_3_accuracy did not improve from 0.95508
Epoch 24/100
8789/8789 [==============================] - 4697s 534ms/step - loss: 0.5959 - categorical_crossentropy: 0.5959 - categorical_accuracy: 0.8414 - top_3_accuracy: 0.9467 - val_loss: 0.5693 - val_categorical_crossentropy: 0.5693 - val_categorical_accuracy: 0.8470 - val_top


Epoch 00043: val_top_3_accuracy did not improve from 0.95620

Epoch 00043: ReduceLROnPlateau reducing learning rate to 6.199045819812455e-05.
Epoch 44/100
8789/8789 [==============================] - 5308s 604ms/step - loss: 0.5376 - categorical_crossentropy: 0.5376 - categorical_accuracy: 0.8556 - top_3_accuracy: 0.9528 - val_loss: 0.5343 - val_categorical_crossentropy: 0.5343 - val_categorical_accuracy: 0.8556 - val_top_3_accuracy: 0.9534

Epoch 00044: val_top_3_accuracy did not improve from 0.95620
Epoch 45/100
8789/8789 [==============================] - 5307s 604ms/step - loss: 0.5372 - categorical_crossentropy: 0.5372 - categorical_accuracy: 0.8559 - top_3_accuracy: 0.9529 - val_loss: 0.5323 - val_categorical_crossentropy: 0.5323 - val_categorical_accuracy: 0.8565 - val_top_3_accuracy: 0.9536

Epoch 00045: val_top_3_accuracy did not improve from 0.95620
Epoch 46/100
8789/8789 [==============================] - 5306s 604ms/step - loss: 0.5251 - categorical_crossentropy: 0.5251 - 

KeyboardInterrupt: 

In [48]:
# run the validation
import gc
# gpu_model.load_weights('xception_model.h5') # best run - otherwise choose xception_model_last.h5
gpu_model.load_weights('xception_model_last.h5')
valid_batch_size = 30000
valid_data = pd.read_csv(os.path.join('shuffled_train_20/train_k19.csv.gz'))

num_batch = np.ceil(len(valid_data)/valid_batch_size) # can override to reduce the number of validation
counter = 0
nrows = 0
pred_valid_array = []

while True:
    batch_files = valid_data[valid_batch_size*counter:valid_batch_size*(counter+1)]
    
    x_valid = df_to_image_array_xd(batch_files, size)
    p_valid = gpu_model.predict(x_valid, batch_size=1024, verbose=1)
    
    if counter == 0:
        pred_valid_array = p_valid
    else:
        pred_valid_array = np.concatenate((pred_valid_array, p_valid), axis = 0)
    print('Pred shape: {}'.format(pred_valid_array.shape))
    
    x_valid = []
    gc.collect()
    
    counter += 1
    nrows += len(batch_files)
    if counter >= num_batch:
            break

print(nrows)
map3 = mapk(valid_data[['y']].values, preds2catids(pred_valid_array).values)
print('Map3: {:.3f}'.format(map3))

/home/wilan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:228: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


30000/30000 [==============================] - 19s 634us/step
Pred shape: (30000, 340)


0

30000/30000 [==============================] - 18s 614us/step
Pred shape: (60000, 340)


0

...

30000/30000 [==============================] - 18s 598us/step
Pred shape: (2460000, 340)


0

26554/26554 [==============================] - 16s 617us/step
Pred shape: (2486554, 340)


0

2486554
Map3: 0.892


In [49]:
test_batch_size = 10000
test = pd.read_csv(os.path.join('test_simplified.csv'))

num_batch = np.ceil(len(test)/test_batch_size)
counter = 0
nrows = 0
pred_array = []

while True:
    batch_files = test[test_batch_size*counter:test_batch_size*(counter+1)]
    
    x_test = df_to_image_array_xd(batch_files, size)
    p_test = gpu_model.predict(x_test, batch_size=256, verbose=1)
    
    if counter == 0:
        pred_array = p_test
    else:
        pred_array = np.concatenate((pred_array, p_test), axis = 0)
    print('Pred shape: {}'.format(pred_array.shape))
    
    x_valid = []
    gc.collect()
    
    counter += 1
    nrows += len(batch_files)
    if counter >= num_batch:
            break

print(nrows)

/home/wilan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:228: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


10000/10000 [==============================] - 7s 688us/step
Pred shape: (10000, 340)


0

10000/10000 [==============================] - 7s 678us/step
Pred shape: (20000, 340)


0

...

10000/10000 [==============================] - 7s 669us/step
Pred shape: (110000, 340)


0

2199/2199 [==============================] - 1s 677us/step
Pred shape: (112199, 340)


0

112199


In [51]:
# standard code from kernel to convert to submission file
top3 = preds2catids(pred_array)
top3.head()
top3.shape

def list_all_categories():
    files = os.listdir(os.path.join(INPUT_DIR, 'train_simplified'))
    return sorted([f2cat(f) for f in files], key=str.lower)

cats = list_all_categories()
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(cats)}
top3cats = top3.replace(id2cat)
top3cats.head()
top3cats.shape

,a,b,c
0,234,281,29
1,144,36,226
2,305,62,110
3,187,111,82
4,113,56,112


(112199, 3)

,a,b,c
0,radio,stereo,binoculars
1,hockey_puck,bottlecap,pool
2,The_Great_Wall_of_China,castle,fence
3,mountain,finger,crayon
4,fireplace,campfire,fire_hydrant


(112199, 3)

In [50]:
### This section is for ensemble output only
def preds2catids_pr(predictions, k=5):
    topk = pd.DataFrame(np.argsort(-predictions, axis=1)[:, :k], columns=[chr(ord('a')+x) for x in range(k)])
    
    cats = list_all_categories()
    id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(cats)}
    topcats = topk.replace(id2cat)
    
    p_pr = pd.DataFrame(-np.sort(-predictions, axis=1)[:, :k])
    p_all = pd.concat([topcats, p_pr], axis=1)
    
    # construct the word
    p_all['word'] = ""
    
    for l in range(k):
        p_all['word'] += (p_all[chr(ord('a')+l)] + " " + p_all[l].astype(str) + " ")
    
    # reuse the test CSV for ids
    test_df = pd.read_csv(os.path.join('test_simplified.csv'))
    test_df['word'] = p_all['word']
    
    return test_df 

x = preds2catids_pr(pred_array)
x.head()

# generate the top 5 pr CSV
x[['key_id', 'word']].to_csv('xception128_top5_last.csv', index = False)

,key_id,countrycode,drawing,word
0,9000003627287624,DE,"[[[17, 18, 20, 25, 137, 174, 242, 249, 251, 25...",radio 0.8984669 stereo 0.10153251 binoculars 2...
1,9000010688666847,UA,"[[[174, 145, 106, 38, 11, 4, 4, 15, 29, 78, 16...",hockey_puck 0.9542305 bottlecap 0.03710187 poo...
2,9000023642890129,BG,"[[[0, 12, 14, 17, 16, 24, 55, 57, 60, 79, 82, ...",The_Great_Wall_of_China 0.9422603 castle 0.055...
3,9000038588854897,US,"[[[0, 9, 23, 40, 54, 60, 81, 105, 123, 167, 20...",mountain 0.99999714 crayon 6.417032e-07 finger...
4,9000052667981386,AR,"[[[87, 82, 71, 63, 66, 92, 96, 95], [220, 218,...",fireplace 0.58886385 campfire 0.4010205 fire_h...


In [52]:
test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test[['key_id', 'word']]
submission.to_csv('benchmark_submission_map3_{}_v3_ens.csv'.format(int(map3 * 10**4)), index=False)
submission.head()
submission.shape

,key_id,word
0,9000003627287624,radio stereo binoculars
1,9000010688666847,hockey_puck bottlecap pool
2,9000023642890129,The_Great_Wall_of_China castle fence
3,9000038588854897,mountain finger crayon
4,9000052667981386,fireplace campfire fire_hydrant


(112199, 2)

# Xception 256

In [51]:
base_model = Xception(input_shape = (256, 256, 3), include_top = False, weights = 'imagenet')
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(340, activation = 'softmax')(x)
model = Model(inputs = base_model.input, outputs = x)
gpu_model = multi_gpu_model(model, 2)
print(gpu_model.summary())
gpu_model.load_weights('xception_model256_cur.h5')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
lambda_5 (Lambda)               (None, 256, 256, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
lambda_6 (Lambda)               (None, 256, 256, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
model_5 (Model)                 (None, 340)          21558140    lambda_5[0][0]                   
                                                                 lambda_6[0][0]                   
__________

In [ ]:
### Xception 256
from keras.callbacks import CSVLogger

EPOCHS = 100
TOTAL_TRAIN = 2250000
BATCH_SIZE = 32 * 2
STEPS = TOTAL_TRAIN // BATCH_SIZE

train_datagen = image_generator_xd(size=size, batchsize=BATCH_SIZE, ks=range(NCSVS - 1))
valid_datagen = image_generator_xd(size=size, batchsize=BATCH_SIZE, ks=NCSVS - 1)

def step_decay_exp(epoch):
    initial_lrate = 0.00075
    drop = 0.5 # drop by how much? 0.5 = half each time
    epochs_drop = 7.0 # drop how many x epochs
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return max(1e-6, lrate)

callbacks = [
    ModelCheckpoint('xception_model256.h5', monitor = 'val_top_3_accuracy', mode = 'max', save_best_only = True, verbose = 1),
    ModelCheckpoint('xception_model256_cur.h5', monitor = 'val_top_3_accuracy', mode = 'max', save_best_only = False, verbose = 0),
    ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.66, patience=3,
                      min_lr = 0.000001, 
#                       min_delta=0.005, 
                      mode='max', 
                      cooldown=3, verbose=1),
#     LearningRateScheduler(step_decay_exp),
    CSVLogger('xception_log256.csv', append=True, separator=';')
]

# sgdOptimiser = SGD(lr = 0.001,  momentum=0.9, decay = 0.0001,  nesterov=True)

gpu_model.compile(optimizer=Adam(lr=0.00015), loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])

gpu_model.load_weights('xception_model256_cur.h5')

gpu_model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data = valid_datagen, validation_steps = STEPS // 8,
    callbacks = callbacks
)


In [52]:
test_batch_size = 5000
test = pd.read_csv(os.path.join('test_simplified.csv'))

num_batch = np.ceil(len(test)/test_batch_size)
counter = 0
nrows = 0
pred_array = []

while True:
    batch_files = test[test_batch_size*counter:test_batch_size*(counter+1)]
    
    x_test = df_to_image_array_xd(batch_files, 256)
    p_test = gpu_model.predict(x_test, batch_size=256, verbose=1)
    
    if counter == 0:
        pred_array = p_test
    else:
        pred_array = np.concatenate((pred_array, p_test), axis = 0)
    print('Pred shape: {}'.format(pred_array.shape))
    
    x_valid = []
    gc.collect()
    
    counter += 1
    nrows += len(batch_files)
    if counter >= num_batch:
            break

print(nrows)

/home/wilan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:228: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


5000/5000 [==============================] - 15s 3ms/step
Pred shape: (5000, 340)


0

5000/5000 [==============================] - 13s 3ms/step
Pred shape: (10000, 340)


0

...

5000/5000 [==============================] - 12s 2ms/step
Pred shape: (110000, 340)


0

2199/2199 [==============================] - 6s 3ms/step
Pred shape: (112199, 340)


0

112199


In [53]:
x = preds2catids_pr(pred_array)
x.head()

# generate the top 5 pr CSV
x[['key_id', 'word']].to_csv('xception256_top5_last.csv', index = False)

,key_id,countrycode,drawing,word
0,9000003627287624,DE,"[[[17, 18, 20, 25, 137, 174, 242, 249, 251, 25...",radio 0.9045055 stereo 0.09549432 binoculars 1...
1,9000010688666847,UA,"[[[174, 145, 106, 38, 11, 4, 4, 15, 29, 78, 16...",hockey_puck 0.92658585 bottlecap 0.05971142 po...
2,9000023642890129,BG,"[[[0, 12, 14, 17, 16, 24, 55, 57, 60, 79, 82, ...",The_Great_Wall_of_China 0.93313104 castle 0.06...
3,9000038588854897,US,"[[[0, 9, 23, 40, 54, 60, 81, 105, 123, 167, 20...",mountain 0.9999871 crayon 4.4550693e-06 finger...
4,9000052667981386,AR,"[[[87, 82, 71, 63, 66, 92, 96, 95], [220, 218,...",campfire 0.6797234 fireplace 0.31749898 fire_h...
